In [1]:
from underthesea import pos_tag, chunk, word_tokenize
from nltk import RegexpParser, Tree
import pandas as pd
import numpy as np

In [34]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP("./VnCoreNLP-master/VnCoreNLP-1.1.1.jar", annotators="wseg,pos", max_heap_size='-Xmx500m') 

def vnpos(text): # part-of-speech tagging
    annotated_text = annotator.annotate(text)
    pos = []
    for dic in annotated_text['sentences'][0]:
#         if dic['posTag'] not in ['L','M','E','C','Cc','I','T','Y','Z','X','CH']:
          pos.append(tuple([dic['form'], dic['posTag']]))
        
    return pos

In [2]:
fix = pd.read_csv('./data/noise.csv')['Unnamed: 0'].values # index của các bài báo chứa data bị lỗi

In [3]:
def preprocess(s): #loại bỏ punctuation
    return s.replace(':','').replace(',','').replace('“','').replace('”','').replace('(','').replace(')','').replace('-','')

In [4]:
df = pd.read_csv('./data/filtered_news.csv').drop('Unnamed: 0', axis=1)
text = df.NewsFullContent.apply(lambda s: preprocess(str(s))).replace('', np.nan)
# text = df.NewsFullContent.replace('', np.nan)
text = text[text.notna()] # loại bỏ các dòng NewsFullContent rỗng
text.head()

0    . tên người thực hiện giao dịch phạm nhật vượn...
1    tổ chức thực hiện giao dịch công ty cổ phần cơ...
2    số lượng tại thời điểm  công văn cvssiho xác n...
3    đơn vị tỷ vnđ khoản mục quý iii tháng giá trị ...
4    đơn vị vnd stt chỉ tiêu quý  lũy kế tháng đầu ...
Name: NewsFullContent, dtype: object

In [5]:
text = text.reset_index().drop('index', axis=1).drop(index=fix, axis=0).reset_index().drop('index', axis=1) #loại bỏ các bài báo chứa data bị lỗi
print(text)

                                         NewsFullContent
0      . tên người thực hiện giao dịch phạm nhật vượn...
1      tổ chức thực hiện giao dịch công ty cổ phần cơ...
2      số lượng tại thời điểm  công văn cvssiho xác n...
3      đơn vị tỷ vnđ khoản mục quý iii tháng giá trị ...
4      đơn vị vnd stt chỉ tiêu quý  lũy kế tháng đầu ...
...                                                  ...
15687  cổ phiếu ctg  có dấu hiệu mua chủ động với khố...
15688  cổ phiếu ssi  có dấu hiệu bán chủ động với khố...
15689  cổ phiếu pnj  có dấu hiệu bán chủ động với khố...
15690  chốt phiên giao dịch ngày  cổ phiếu vcb dừng ở...
15691  hội đồng quản trị công ty cổ phần tập đoàn mas...

[15692 rows x 1 columns]


# Candidate keyword extraction

In [37]:
with open('./data/text.txt', 'r', encoding="utf8") as f:
    text = f.read()
print(text)

Tổng công ty Điện lực Dầu khí Việt Nam (PV Power – POW) đang thực hiện các bước để thực hiện thoái vốn tại CTCP Máy - Thiết bị Dầu khí (PV Machino – PVM). PV Power đang thuê định giá và tư vấn thoái vốn, ông Hồ Công Kỳ cho biết tại ĐHĐCĐ bất thường tổ chức ngày. "Trên cơ sở kết quả tư vấn, HĐQT sẽ xem xét phương thức thoái vốn cụ thể. Chúng tôi sẽ triển khai minh bạch và tuân thủ quy định pháp luật. Các tổ chức định giá sẽ phải nắm bắt đầy đủ tình hình tài sản của PVM, kể cả các bất động sản", ông Kỳ nói thêm. Việc thoái vốn tại PV Machino được nhiều cổ đông PV Power quan tâm tại ĐHĐCĐ bất thường hôm nay. Bản thân PV Power đang là công ty mẹ, sở hữu cổ phần, tương ứng gần tỷ đồng trên vốn điều lệ. Ông Hồ Công Kỳ cho biết, PV Power mua PVM dưới mệnh giá, thoái vốn lợi nhuận ít nhất vài trăm tỷ (chia sẻ bên thềm đại hội). Chủ tịch POW không hẳn không có lý khi giá trên sàn của PVM hiện đã vượt mức đồng. Nhưng cổ đông POW đang kỳ vọng nhiều hơn dựa trên những tài sản của PV Machino hiện c

In [44]:
text = text.replace("(","").replace(")","").replace('"','').replace('– ','').replace('- ','')
text

'Tổng công ty Điện lực Dầu khí Việt Nam PV Power POW đang thực hiện các bước để thực hiện thoái vốn tại CTCP Máy Thiết bị Dầu khí PV Machino PVM. PV Power đang thuê định giá và tư vấn thoái vốn, ông Hồ Công Kỳ cho biết tại ĐHĐCĐ bất thường tổ chức ngày. Trên cơ sở kết quả tư vấn, HĐQT sẽ xem xét phương thức thoái vốn cụ thể. Chúng tôi sẽ triển khai minh bạch và tuân thủ quy định pháp luật. Các tổ chức định giá sẽ phải nắm bắt đầy đủ tình hình tài sản của PVM, kể cả các bất động sản, ông Kỳ nói thêm. Việc thoái vốn tại PV Machino được nhiều cổ đông PV Power quan tâm tại ĐHĐCĐ bất thường hôm nay. Bản thân PV Power đang là công ty mẹ, sở hữu cổ phần, tương ứng gần tỷ đồng trên vốn điều lệ. Ông Hồ Công Kỳ cho biết, PV Power mua PVM dưới mệnh giá, thoái vốn lợi nhuận ít nhất vài trăm tỷ chia sẻ bên thềm đại hội. Chủ tịch POW không hẳn không có lý khi giá trên sàn của PVM hiện đã vượt mức đồng. Nhưng cổ đông POW đang kỳ vọng nhiều hơn dựa trên những tài sản của PV Machino hiện có, bao gồm bấ

In [49]:
len(word_tokenize(text))

356

In [41]:
grammar = r"""
                NP: {(<N.*>+ <A>* <E>)? <N.*>+ <A>*}
                VP: {<V.*>+ <L>* <N.*>* <R>* <A>*}
           """ 
# VP: {<V.*>((<N.*>+ <A>* <E>)? <N.*>+ <A>*)?<R>*<A>*}
chunker = RegexpParser(grammar)

In [35]:
def get_continuous_chunks(text, chunk_func=chunker.parse):
#     text_tokens = word_tokenize(text)
#     tokens_without_sw = [word for word in text_tokens if not word in stopwords]
#     filter_sent = (" ").join(tokens_without_sw)
#     pos = pos_tag(text)
#     pos_filter = [(a,b) for (a,b) in pos if b not in ['L','M','E','C','Cc','I','T','Y','Z','X','CH']]
    chunked = chunk_func(vnpos(text))
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
        if type(subtree) == Tree:
            current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue

    return continuous_chunk

In [ ]:
candidate_keyword = text.NewsFullContent.apply(lambda s: get_continuous_chunks(s)) 

In [40]:
vnpos(text)

[('Tổng_công_ty', 'N'),
 ('Điện_lực', 'N'),
 ('Dầu_khí', 'N'),
 ('Việt_Nam', 'Np'),
 ('PV', 'Ny'),
 ('Power_POW', 'Np'),
 ('đang', 'R'),
 ('thực_hiện', 'V'),
 ('các', 'L'),
 ('bước', 'N'),
 ('để', 'E'),
 ('thực_hiện', 'V'),
 ('thoái', 'V'),
 ('vốn', 'N'),
 ('tại', 'E'),
 ('CTCP', 'Ny'),
 ('Máy', 'Np'),
 ('-', 'CH'),
 ('Thiết_bị', 'V'),
 ('Dầu_khí', 'N'),
 ('PV', 'Ny'),
 ('Machino_PVM', 'Np'),
 ('.', 'CH')]

In [42]:
get_continuous_chunks(text)

['Tổng_công_ty Điện_lực Dầu_khí Việt_Nam PV Power_POW',
 'thực_hiện',
 'bước',
 'thực_hiện thoái vốn tại CTCP Máy',
 'Thiết_bị Dầu_khí PV Machino_PVM']